<a href="https://colab.research.google.com/github/AmartyaCodes/DogBreedClassification-/blob/main/DogBreedClassificationRestAPI.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# CREATING A FLASK REST API


In [26]:
!pip install flask-ngrok

In [27]:
pip install flask gevent requests pillow

## IMPORTING NECESSARY LIBRARIES

In [28]:
import tensorflow as tf
from __future__ import division, print_function
# coding=utf-8
import sys
import os
import glob
import re
import numpy as np

# Keras
from keras.applications.imagenet_utils import preprocess_input, decode_predictions
from keras.models import load_model
from keras.preprocessing import image

# Flask utils
from flask import Flask, redirect, url_for, request, render_template
from werkzeug.utils import secure_filename
from gevent.pywsgi import WSGIServer

In [29]:
import os
os.mkdir('uploads')

## USING RESNET 50 MODEL TO PREDICT

In [30]:
# import the necessary packages
from tensorflow.keras.applications import ResNet50
from tensorflow.keras.preprocessing.image import img_to_array
from tensorflow.keras.applications import imagenet_utils
from PIL import Image
import numpy as np
import flask
import io
from flask_ngrok import run_with_ngrok
from flask import Flask
# initialize our Flask application and the Keras model
app = flask.Flask(__name__)
run_with_ngrok(app)
#MODEL_PATH = '/content/drive/MyDrive/DogClassificationByResNet2.h5'

# Load your trained model
# model = load_model(MODEL_PATH,custom_objects={"custom_metric": custom_metric})
model = ResNet50(weights="imagenet")

# USING STATIC FILES AND TEMPLATES

In [31]:
from zipfile import ZipFile
file_name = "/content/templates.zip"

with ZipFile(file_name , 'r') as zip:
  zip.extractall()
  print('Done')

Done


In [32]:
from zipfile import ZipFile
file_name = "/content/static.zip"

with ZipFile(file_name , 'r') as zip:
  zip.extractall()
  print('Done')

Done


# Defining the model and workflows

In [33]:
def model_predict(img_path, model):
    img = image.load_img(img_path, target_size=(224, 224))

    # Preprocessing the image
    x = image.img_to_array(img)
    # x = np.true_divide(x, 255)
    x = np.expand_dims(x, axis=0)

    # Be careful how your trained model deals with the input
    # otherwise, it won't make correct prediction!
    x = preprocess_input(x, mode='caffe')

    preds = model.predict(x)
    return preds

In [34]:
@app.route('/', methods=['GET'])
def index():
    # Main page
    return render_template('index.html')


In [35]:
@app.route('/predict', methods=['GET', 'POST'])
def upload():
    if request.method == 'POST':
        # Get the file from post request
        f = request.files['file']

        # Save the file to ./uploads
        basepath = os.path.dirname('/content/')
        file_path = os.path.join(
            basepath, 'uploads', secure_filename(f.filename))
        f.save(file_path)

        # Make prediction
        preds = model_predict(file_path, model)

        # Process your result for human
        # pred_class = preds.argmax(axis=-1)            # Simple argmax
        pred_class = decode_predictions(preds, top=1)   # ImageNet Decode
        result = str(pred_class[0][0][1])               # Convert to string
        return result
    return None

In [36]:
if __name__ == '__main__':
    app.run()

 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://127.0.0.1:5000/ (Press CTRL+C to quit)


 * Running on http://25488342f7a5.ngrok.io
 * Traffic stats available on http://127.0.0.1:4040


127.0.0.1 - - [13/Jun/2021 18:41:16] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [13/Jun/2021 18:41:17] "GET /static/css/main.css HTTP/1.1" 200 -
127.0.0.1 - - [13/Jun/2021 18:41:17] "GET /static/js/main.js HTTP/1.1" 200 -
127.0.0.1 - - [13/Jun/2021 18:41:18] "GET /favicon.ico HTTP/1.1" 404 -
127.0.0.1 - - [13/Jun/2021 18:41:18] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [13/Jun/2021 18:41:20] "GET /static/css/main.css HTTP/1.1" 200 -
127.0.0.1 - - [13/Jun/2021 18:41:20] "GET /static/js/main.js HTTP/1.1" 200 -
127.0.0.1 - - [13/Jun/2021 18:41:38] "POST /predict HTTP/1.1" 200 -
